# 【数学復習】手計算で理解する固有値分解

PCAの核心である $\Sigma v = \lambda v$ という式が、具体的にどうやって「軸」と「長さ」を見つけているのか、簡単な行列を使って計算してみます。

## 1. 問題設定

あるデータの共分散行列 $\Sigma$ が、以下の $2 \times 2$ 行列になったとします。

$$
\Sigma = 
\begin{pmatrix}
2 & 1 \\
1 & 2
\end{pmatrix}
$$

* **対角成分 ($2, 2$)**: $x$ と $y$ それぞれの分散。どちらも同じくらい散らばっている。
* **非対角成分 ($1, 1$)**: 共分散。$x$ が増えると $y$ も増える（正の相関）。
* **形**: 右上がりの楕円形データです。

この行列 $\Sigma$ を固有値分解して、**回転しても向きが変わらない特別なベクトル（固有ベクトル）**を見つけます。

---

## 2. 固有値 $\lambda$ を求める

定義式 $\Sigma v = \lambda v$ を変形すると、$(\Sigma - \lambda I)v = 0$ となります。
$v$ がゼロベクトル以外（意味のある方向）であるためには、係数行列の行列式が $0$ になる必要があります。

**特性方程式**:
$$
\det(\Sigma - \lambda I) = 0
$$

具体的な数字を当てはめます。

$$
\det \begin{pmatrix}
2 - \lambda & 1 \\
1 & 2 - \lambda
\end{pmatrix} = 0
$$

たすき掛けで計算します（$ad - bc$）。

$$
(2 - \lambda)(2 - \lambda) - (1 \times 1) = 0 \\
(4 - 4\lambda + \lambda^2) - 1 = 0 \\
\lambda^2 - 4\lambda + 3 = 0
$$

因数分解します。

$$
(\lambda - 3)(\lambda - 1) = 0
$$

よって、固有値は **$\lambda = 3$** と **$\lambda = 1$** です。
* PCA的に言うと、**第1主成分の分散（パワー）が 3**、**第2主成分の分散が 1** ということになります。

---

## 3. 固有ベクトル $v$ を求める

それぞれの固有値（パワー）に対応する固有ベクトル（向き）を計算します。

### ケース1: $\lambda = 3$ のとき（主成分）

元の式 $(\Sigma - \lambda I)v = 0$ に $\lambda = 3$ を代入します。

$$
\begin{pmatrix}
2 - 3 & 1 \\
1 & 2 - 3
\end{pmatrix}
\begin{pmatrix}
x \\
y
\end{pmatrix}
=
\begin{pmatrix}
0 \\
0
\end{pmatrix}
$$

$$
\begin{pmatrix}
-1 & 1 \\
1 & -1
\end{pmatrix}
\begin{pmatrix}
x \\
y
\end{pmatrix}
=
\begin{pmatrix}
0 \\
0
\end{pmatrix}
$$

これを行列の掛け算として展開すると：
$$
-x + y = 0 \quad \Rightarrow \quad y = x
$$

つまり、$x$ と $y$ が同じ値であれば何でもOKです。
長さを1に正規化すると、
$$
v_1 = \begin{pmatrix} \frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} \end{pmatrix} \quad (\text{右上45度の方向})
$$

### ケース2: $\lambda = 1$ のとき（第2成分）

同様に $\lambda = 1$ を代入します。

$$
\begin{pmatrix}
2 - 1 & 1 \\
1 & 2 - 1
\end{pmatrix}
\begin{pmatrix}
x \\
y
\end{pmatrix}
=
\begin{pmatrix}
1 & 1 \\
1 & 1
\end{pmatrix}
\begin{pmatrix}
x \\
y
\end{pmatrix}
= 0
$$

$$
x + y = 0 \quad \Rightarrow \quad y = -x
$$

長さを1に正規化すると、
$$
v_2 = \begin{pmatrix} -\frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} \end{pmatrix} \quad (\text{左上135度の方向})
$$

---

## 4. 結論：何がわかったか？

この計算結果は、データの構造を完全に説明しています。

1.  **最も分散が大きい方向 ($v_1$)**: 右上45度方向（$\nearrow$）。
2.  **その分散の大きさ ($\lambda_1$)**: $3$。
3.  **次に分散が大きい方向 ($v_2$)**: 左上135度方向（$\nwarrow$）。$v_1$ と直角に交わっている。
4.  **その分散の大きさ ($\lambda_2$)**: $1$。

$$
\text{元の共分散行列} \begin{pmatrix} 2 & 1 \\ 1 & 2 \end{pmatrix}
\xrightarrow{\text{固有値分解}}
\text{「斜め45度に、パワー3で伸びている」}
$$

これが、Pythonの `tf.linalg.eigh` が内部で高速に行っている計算の正体です。

## 【数学的証明】なぜ固有値分解で主成分が求まるのか？

「データの分散を最大化する軸を見つける」というPCAの目的が、「共分散行列の固有値問題を解く」ことに帰着する理由を証明する。

### 1. 問題の定式化
ある単位ベクトル $v$ ($||v||=1$) にデータを射影したとき、その射影されたデータの分散 $S^2$ は次のように表される。

$$
S^2 = \frac{1}{N-1} \sum_{i=1}^{N} (x_i^T v)^2 = v^T \left( \frac{1}{N-1} X^T X \right) v = v^T \Sigma v
$$

ここで、$\Sigma$ は共分散行列である。
つまり、**「$v^T \Sigma v$ を最大化する $v$ を見つけること」** が目的となる。

### 2. ラグランジュの未定乗数法
制約条件 $v^T v = 1$ の下で、目的関数 $f(v) = v^T \Sigma v$ を最大化する問題を解く。
ラグランジュ関数 $L$ を以下のように置く。

$$
L(v, \lambda) = v^T \Sigma v - \lambda (v^T v - 1)
$$

これをベクトル $v$ で微分して $0$ と置く。

$$
\frac{\partial L}{\partial v} = 2 \Sigma v - 2 \lambda v = 0
$$

### 3. 固有値問題への帰着
上の式を整理すると、以下の関係式が得られる。

$$
\Sigma v = \lambda v
$$

これはまさに **「共分散行列 $\Sigma$ の固有値問題」** そのものである。

### 4. 分散の正体
このとき、最大化したい分散 $S^2$ （目的関数）の値はどうなるか。
固有値問題の定義式 $\Sigma v = \lambda v$ を、分散の式 $v^T \Sigma v$ に代入する。

$$
S^2 = v^T (\Sigma v) = v^T (\lambda v) = \lambda (v^T v) = \lambda \cdot 1 = \lambda
$$

### 結論
* 射影後の分散 $S^2$ は、**固有値 $\lambda$ そのもの** になる。
* 分散を最大化したければ、**最大の固有値 $\lambda_{max}$** を選び、その時の **固有ベクトル $v$** を軸とすればよい。